In [1]:
import numpy as np

from probabilistic_classifier.dataset import create_joint_marginal_dataset
from probabilistic_classifier.estimate import estimate_mi_for_binary_classification
from probabilistic_classifier.train import train_binary_classifier_v2

In [2]:
# create the basis dataset
mean = [0, 0]
cov = [[1, 0.8],
       [0.8, 1]]
num_of_samples = 800000
dataset = np.random.multivariate_normal(mean=mean, cov=cov, size=num_of_samples)

In [3]:
# create the joint and marginal datasets
x_idx, y_idx = [0], [1]
joint_data, joint_label, marginal_data, marginal_label = create_joint_marginal_dataset(dataset, x_idx, y_idx)
data, label = np.concatenate([joint_data, marginal_data]), np.concatenate([joint_label, marginal_label])
randomize_idx = np.random.permutation(np.arange(2 * num_of_samples))
data, label = data[randomize_idx], label[randomize_idx]

In [4]:
# define model parameters
num_input_features = len(x_idx) + len(y_idx)
hidden_size_arr = [256, 256, 256]
lr = 0.001

In [5]:
num_of_outer_iteration = 20
num_of_inner_iteration = 50
batch_size = 4096

# iterate over many times
outer_running_loss = []
outer_running_loss_avg = []
ldr_estimations = []
dv_estimations = []
nwj_estimations = []

for outer_iter in range(num_of_outer_iteration):
    print('################################################################')
    model, inner_running_loss, inner_running_loss_avg, num_of_joint, num_of_marginal = train_binary_classifier_v2(data, label, num_input_features, hidden_size_arr, lr, num_of_inner_iteration, batch_size, outer_iter, print_progress=False)
    outer_running_loss.append(inner_running_loss)
    outer_running_loss_avg.append(inner_running_loss_avg)
    
    ## estimate cmi
    curr_ldr, curr_dv, curr_nwj = estimate_mi_for_binary_classification(model, joint_data, num_of_joint, marginal_data, num_of_marginal)
    print('trial: {}, ldr: {}, dv: {}, nwj: {}'.format(outer_iter + 1, curr_ldr.item(), curr_dv.item(), curr_nwj.item()))
    print('################################################################\n')
    ldr_estimations.append(curr_ldr.item())
    dv_estimations.append(curr_dv.item())
    nwj_estimations.append(curr_nwj.item())
    
print('final estimations:\n\tldr: {}\n\tdv: {}\n\tnwj: {}'.format(np.mean(ldr_estimations), np.mean(dv_estimations), np.mean(nwj_estimations)))

################################################################
trial: 1, ldr: 0.5337440371513367, dv: 0.5087945461273193, nwj: 0.5084807276725769
################################################################

################################################################
trial: 2, ldr: 0.4757423400878906, dv: 0.5092244148254395, nwj: 0.5086700916290283
################################################################

################################################################
trial: 3, ldr: 0.4825809895992279, dv: 0.5085330009460449, nwj: 0.5081990957260132
################################################################

################################################################
trial: 4, ldr: 0.5264977216720581, dv: 0.5084094405174255, nwj: 0.5082448720932007
################################################################

################################################################
trial: 5, ldr: 0.5064483880996704, dv: 0.5089098215103149, nwj: 0.5089067816734